# Tax Liability Prediction (Regression) — End-to-End ML Notebook 🧾📈

This is a complete **industry-style** machine learning notebook using the synthetic tax dataset.

## 🎯 Objective
Predict **`TaxLiability`** (a continuous numeric value) using demographic, income, and deduction-related features.

## ✅ What’s included
- Data loading + sanity checks
- **Advanced EDA (multi-level)**
- Cleaning + preprocessing (numeric & categorical)
- **Feature engineering**
- Baseline + **6+ regression models**
- Evaluation with **RMSE, MAE, R², MAPE**
- Cross-validation (CV) comparison
- Residual diagnostics (pred vs actual, residual plots)
- Permutation importance (model-agnostic)
- Save best model pipeline for deployment


In [ ]:
# ============================
# 0) Setup
# ============================
DATA_PATH = "tax_synthetic_ml_dataset.csv"  # keep CSV in same folder as this notebook

import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head()

## 1) Data Audit (Quality Checks)
We check:
- types
- missing values
- duplicates
- summary statistics


In [ ]:
df.info()

In [ ]:
# Missing values (should be none for this synthetic dataset, but we still check)
missing = df.isna().sum().sort_values(ascending=False)
missing = missing[missing > 0]
print("Columns with missing:", len(missing))
missing.head(20)

In [ ]:
# Duplicates
dup = df.duplicated().sum()
print("Duplicate rows:", dup)

In [ ]:
df.describe(include="all").T.head(25)

## 2) Define Regression Target
### Target: `TaxLiability`
We drop classification-only label `AuditFlag`.
We also drop `EffectiveTaxRate` because it is derived from the target (`TaxLiability / TotalIncome`), which would cause leakage.


In [ ]:
TARGET = "TaxLiability"

drop_cols = ["AuditFlag", "EffectiveTaxRate"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

X = df.drop(columns=[TARGET])
y = df[TARGET].astype(float)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Target range:", (y.min(), y.max()))

## 3) Advanced EDA (Level 1): Target Distribution & Skewness

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
y.hist(bins=60)
plt.title("Target Distribution: TaxLiability")
plt.xlabel("TaxLiability")
plt.ylabel("Count")
plt.show()

print("Skewness:", float(y.skew()))
print("Kurtosis:", float(y.kurtosis()))

## 4) Advanced EDA (Level 2): Numeric Distributions & Outliers
We inspect distributions of major numeric variables and identify potential outliers.


In [ ]:
num_cols_all = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_all = X.select_dtypes(exclude=[np.number]).columns.tolist()

major_num = ["Age","AnnualIncome","BusinessIncome","CapitalGains","DeductionsTotal","TaxableIncome"]
major_num = [c for c in major_num if c in X.columns]

for col in major_num:
    plt.figure()
    X[col].hist(bins=50)
    plt.title(f"Distribution: {col}")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.show()

# Quick IQR outlier count (informative only)
def iqr_outlier_count(s):
    q1, q3 = np.percentile(s.dropna(), [25, 75])
    iqr = q3 - q1
    lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr
    return int(((s < lo) | (s > hi)).sum())

outlier_counts = {c: iqr_outlier_count(X[c]) for c in major_num}
pd.Series(outlier_counts).sort_values(ascending=False)

## 5) Advanced EDA (Level 3): Relationships with Target
- Scatter plots for numeric features
- Target mean by categorical features


In [ ]:
# Scatter plots: numeric vs target (sample for speed/clarity)
sample_idx = X.sample(n=min(6000, len(X)), random_state=42).index

for col in major_num:
    plt.figure()
    plt.scatter(X.loc[sample_idx, col], y.loc[sample_idx], alpha=0.25)
    plt.title(f"{col} vs TaxLiability")
    plt.xlabel(col)
    plt.ylabel("TaxLiability")
    plt.show()

In [ ]:
# Categorical: average TaxLiability by category
for col in cat_cols_all:
    stats = df.groupby(col)[TARGET].agg(["mean","median","count"]).sort_values("mean", ascending=False)
    display(stats.head(10))
    
    plt.figure()
    stats["mean"].head(12).plot(kind="bar")
    plt.title(f"Mean TaxLiability by {col} (Top 12)")
    plt.xlabel(col)
    plt.ylabel("Mean TaxLiability")
    plt.show()

## 6) Advanced EDA (Level 4): Correlation (Numeric)
Correlation is useful for redundancy detection and feature selection insight.


In [ ]:
corr = df[num_cols_all + [TARGET]].corr(numeric_only=True)

plt.figure(figsize=(10, 8))
plt.imshow(corr.values, aspect="auto")
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.index)), corr.index)
plt.title("Correlation Heatmap (Numeric)")
plt.tight_layout()
plt.show()

## 7) Feature Engineering (Professional)
We create additional meaningful features:
- `TotalIncome` = Annual + Business + CapitalGains
- `BusinessIncomeRatio` = Business / (Annual + 1)
- `DeductionsRatio` = DeductionsTotal / (TotalIncome + 1)
- `IncomePerDependent` = TotalIncome / (Dependents + 1)
- `LogTotalIncome` = log1p(TotalIncome) to reduce skewness


In [ ]:
df_fe = df.copy()

df_fe["TotalIncome"] = df_fe["AnnualIncome"] + df_fe["BusinessIncome"] + df_fe["CapitalGains"]
df_fe["BusinessIncomeRatio"] = df_fe["BusinessIncome"] / (df_fe["AnnualIncome"] + 1.0)
df_fe["DeductionsRatio"] = df_fe["DeductionsTotal"] / (df_fe["TotalIncome"] + 1.0)
df_fe["IncomePerDependent"] = df_fe["TotalIncome"] / (df_fe["Dependents"] + 1.0)
df_fe["LogTotalIncome"] = np.log1p(df_fe["TotalIncome"])

X = df_fe.drop(columns=[TARGET])
y = df_fe[TARGET].astype(float)

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

X.head()

## 8) Train/Validation/Test Split
We use:
- Train (70%)
- Validation (15%)
- Test (15%)

Validation helps model selection without touching the final test set.


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

## 9) Preprocessing Pipeline
- Numeric: median imputation + scaling
- Categorical: most_frequent imputation + one-hot encoding


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(transformers=[
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols),
])

## 10) Evaluation Metrics (Regression)
We compute:
- **RMSE** (lower is better)
- **MAE** (lower is better)
- **R²** (higher is better)
- **MAPE** (lower is better; careful when y≈0)


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def regression_metrics(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    # avoid division by tiny values for MAPE
    denom = np.clip(np.abs(y_true), 1.0, None)
    mape = np.mean(np.abs((y_true - y_pred) / denom)) * 100
    return {"RMSE": rmse, "MAE": mae, "R2": r2, "MAPE_%": mape}

## 11) Train 6+ Models (Baseline to Advanced)
Models:
1. Linear Regression (baseline)
2. Ridge
3. Lasso
4. ElasticNet
5. RandomForestRegressor
6. ExtraTreesRegressor
7. HistGradientBoostingRegressor

We compare on the **validation set**.


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "Lasso": Lasso(alpha=0.0008, random_state=42, max_iter=20000),
    "ElasticNet": ElasticNet(alpha=0.001, l1_ratio=0.35, random_state=42, max_iter=20000),
    "RandomForest": RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=600, random_state=42, n_jobs=-1),
    "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42)
}

val_results = []
trained = {}

for name, model in models.items():
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    pipe.fit(X_train, y_train)
    pred_val = pipe.predict(X_val)
    m = regression_metrics(y_val, pred_val)
    m["Model"] = name
    val_results.append(m)
    trained[name] = pipe
    print(name, m)

val_df = pd.DataFrame(val_results).set_index("Model").sort_values(["RMSE","MAE"])
val_df

## 12) Cross-Validation (CV) Score for Top Models
We use 5-fold CV RMSE to confirm stability.


In [ ]:
from sklearn.model_selection import KFold, cross_val_score

top_models = val_df.head(4).index.tolist()
cv = KFold(n_splits=5, shuffle=True, random_state=42)

cv_rows = []
for name in top_models:
    pipe = trained[name]
    # negative RMSE (scikit returns negative for loss metrics)
    scores = cross_val_score(pipe, X_train, y_train, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=-1)
    cv_rows.append({
        "Model": name,
        "CV_RMSE_mean": (-scores).mean(),
        "CV_RMSE_std": (-scores).std()
    })

cv_df = pd.DataFrame(cv_rows).set_index("Model").sort_values("CV_RMSE_mean")
cv_df

## 13) Final Model Selection + Test Evaluation
We pick the best model by **validation RMSE**, refit on **Train+Val**, then evaluate on **Test**.


In [ ]:
best_name = val_df.index[0]
print("Best model by validation RMSE:", best_name)

best_pipe = Pipeline(steps=[("preprocess", preprocess), ("model", models[best_name])])

# Refit on Train+Val
X_trval = pd.concat([X_train, X_val], axis=0)
y_trval = pd.concat([y_train, y_val], axis=0)

best_pipe.fit(X_trval, y_trval)

pred_test = best_pipe.predict(X_test)
test_metrics = regression_metrics(y_test, pred_test)
test_metrics

## 14) Diagnostics (Professional Plots)
- Predicted vs Actual
- Residual distribution
- Residuals vs Predicted (heteroscedasticity check)


In [ ]:
# Predicted vs Actual
plt.figure()
plt.scatter(y_test, pred_test, alpha=0.3)
plt.title(f"Predicted vs Actual (Test) — {best_name}")
plt.xlabel("Actual TaxLiability")
plt.ylabel("Predicted TaxLiability")
plt.show()

# Residuals
resid = y_test - pred_test

plt.figure()
resid.hist(bins=60)
plt.title("Residual Distribution (Test)")
plt.xlabel("Residual (Actual - Pred)")
plt.ylabel("Count")
plt.show()

plt.figure()
plt.scatter(pred_test, resid, alpha=0.3)
plt.axhline(0, color="red")
plt.title("Residuals vs Predicted (Test)")
plt.xlabel("Predicted TaxLiability")
plt.ylabel("Residual")
plt.show()

## 15) Permutation Importance (Model-Agnostic)
Shows which input features matter most (works with the full pipeline).


In [ ]:
from sklearn.inspection import permutation_importance

# Use a subset for speed
X_imp = X_test.sample(n=min(5000, len(X_test)), random_state=42)
y_imp = y_test.loc[X_imp.index]

r = permutation_importance(best_pipe, X_imp, y_imp, n_repeats=5, random_state=42, n_jobs=-1, scoring="neg_root_mean_squared_error")

# Build final feature names after OHE
ohe = best_pipe.named_steps["preprocess"].named_transformers_["cat"].named_steps["ohe"]
cat_feature_names = ohe.get_feature_names_out(cat_cols).tolist()
feature_names = num_cols + cat_feature_names

imp_df = pd.DataFrame({
    "feature": feature_names,
    "importance_mean": r.importances_mean,
    "importance_std": r.importances_std
}).sort_values("importance_mean", ascending=False)

imp_df.head(20)

In [ ]:
# Plot top 20 importances
topk = imp_df.head(20).iloc[::-1]

plt.figure(figsize=(8, 7))
plt.barh(topk["feature"], topk["importance_mean"])
plt.title("Top 20 Permutation Importances (RMSE impact)")
plt.xlabel("Importance (mean)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()

## 16) Save Final Pipeline (Deployment-Ready)
This `.joblib` contains preprocessing + model in one object.


In [ ]:
import joblib

ARTIFACT_PATH = "tax_liability_regression_pipeline.joblib"
joblib.dump(best_pipe, ARTIFACT_PATH)
print("Saved:", ARTIFACT_PATH)

## 17) Quick Inference Example
How to predict on a single new sample (dictionary → DataFrame → predict).


In [ ]:
example_row = X_test.iloc[[0]].copy()
pred_example = float(best_pipe.predict(example_row)[0])
print("Example prediction TaxLiability:", pred_example)
example_row

## Conclusion ✅
You built a full **regression** ML pipeline and saved a deployment-ready artifact.

**Best model:** chosen using validation RMSE  
**Final evaluation:** RMSE / MAE / R² / MAPE on test set  
**Explain in viva:** "We transformed the tax dataset into regression to forecast tax liability, using engineered income & deductions features and comparing multiple models with RMSE/MAE/R²."
